In [2]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource
from bokeh.transform import factor_cmap
from bokeh.palettes import Category20
import pandas as pd

# Provided color dictionary
color_dict = {
    "academic_work_day": "lightsteelblue",
    "instructional_day": "lightyellow",
    "convocation": "darkblue",
    "commencement": "lightbrown",
    "finals": "pink",
    "holiday": "white",
    "semester_start": "green",
    "no_class_campus_open": "purple",
    "summer_session": "orange",
    "winter_session": "grey"
}

# Sample data
data = {
    "event": list(color_dict.keys()),
    "color": list(color_dict.values())
}

# Create a DataFrame from the sample data
df = pd.DataFrame(data)

# Create a ColumnDataSource from the DataFrame
source = ColumnDataSource(df)

# Create a figure
p = figure(y_range=df['event'], plot_height=400, plot_width=800, title="Event Colors")

# Add horizontal bars with colored background
p.hbar(y='event', left=0, right=1, height=0.5, source=source,
       line_color="white", fill_color=factor_cmap('event', palette=Category20[10], factors=list(df['event'])))

# Customize plot aesthetics
p.ygrid.grid_line_color = None
p.xaxis.axis_label = "Color"
p.yaxis.axis_label = "Event"
p.title.text_font_size = "20px"
p.title.align = "center"

# Show the plot
output_notebook()  # To display the plot in a Jupyter Notebook
show(p)




Loading BokehJS ...

In [5]:
from bokeh.models import ColumnDataSource, DataTable, TableColumn
from bokeh.plotting import show, output_notebook
import plotly.graph_objects as go


def get_month_data():
    # Define the data
    months = ["Aug-21", "Sep", "Oct", "Nov", "Dec", "Total", "Jan-22", "Feb", "Mar", "Apr", "May", "Total"]
    awd = [0]*12  # Replace with actual data
    id_values = [0]*12  # Replace with actual data

    # Create the table figure
    fig = go.Figure(data=[go.Table(
        header=dict(values=['Month', 'AWD', 'ID']),
        cells=dict(values=[months, awd, id_values]))
    ])

    # Save the figure as PNG
    fig.write_image("table.png")

get_month_data()

Loading BokehJS ...

In [2]:
import calendar
import plotly.graph_objects as go

def get_month_data(year, month):
    num_days = calendar.monthrange(year, month)[1]
    start_day = calendar.monthrange(year, month)[0]
    return num_days, start_day

def generate_calendar_structure(year, month):
    num_days, start_day = get_month_data(year, month)
    calendar_structure = [['' for _ in range(7)] for _ in range(6)]
    day = 1
    for row in range(6):
        for col in range(7):
            if (row == 0 and col >= start_day or row > 0) and day <= num_days:
                calendar_structure[row][col] = day
                day += 1
    return calendar_structure

def plot_calendar(year, month):
    calendar_data = generate_calendar_structure(year, month)
    day_labels = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    
    fig = go.Figure(data=go.Heatmap(
        z=calendar_data,
        x=day_labels,
        y=['Week 1', 'Week 2', 'Week 3', 'Week 4', 'Week 5', 'Week 6'],
        colorscale=[[0, 'white'], [1, 'lightgray']],
        showscale=False,
        text=calendar_data,
        hoverinfo='text'
    ))
    
    fig.update_layout(
        title=f"Calendar for {calendar.month_name[month]} {year}",
        xaxis_nticks=7,
        yaxis_nticks=6,
        yaxis_autorange='reversed'
    )
    
    fig.show()

# Test the function
plot_calendar(2023, 9)



In [7]:
import calendar

def color_weekends(s):
    lines = s.split('\n')
    for i in range(len(lines)):
        # Assuming Sunday is the first column and Saturday is the last
        if i >= 2: # Starting from the line where the dates are
            # Coloring Sunday (start of the line)
            lines[i] = f'\033[91m{lines[i][:3]}\033[0m' + lines[i][3:]
            
            # Coloring Saturday (end of the line)
            lines[i] = lines[i][:-2] + f'\033[91m{lines[i][-2:]}\033[0m'
    return '\n'.join(lines)

cal = calendar.TextCalendar(calendar.SUNDAY) # Sunday as the first day of the week
s = cal.formatmonth(2023, 6)
print(color_weekends(s))

     June 2023
Su Mo Tu We Th Fr Sa
             1  2  3
 4  5  6  7  8  9 10
11 12 13 14 15 16 17
18 19 20 21 22 23 24
25 26 27 28 29 30
[0m


In [24]:
import calendar
from PIL import Image, ImageDraw, ImageFont

def draw_calendar(year, month):
    # Set the first weekday to Sunday
    calendar.setfirstweekday(calendar.SUNDAY)
    cal = calendar.monthcalendar(year, month)
    
    # Adjust day names order to start from Sunday
    day_names = list(calendar.day_abbr)[-1:] + list(calendar.day_abbr)[:-1]  # ["Sun", "Mon", "Tue", ...]

    # Create a blank white image with half the size
    image_width = 350
    image_height = 200
    img = Image.new('RGB', (image_width, image_height), 'white')
    draw = ImageDraw.Draw(img)

    # Load a font
    font_path = "/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf"
    font = ImageFont.truetype(font_path, 15)
    small_font = ImageFont.truetype(font_path, 12)

    # Draw month and year
    title = calendar.month_name[month] + " " + str(year)
    title_bbox = draw.textbbox((0, 0), title, font=font)
    title_w = title_bbox[2] - title_bbox[0]
    title_h = title_bbox[3] - title_bbox[1]
    draw.text(((image_width - title_w) / 2, 5), title, font=font, fill="black")

    # Draw day names
    col_width = image_width / 7
    for idx, day_name in enumerate(day_names):
        color = "red" if idx == 0 or idx == 6 else "black"
        draw.text((idx * col_width + 10, 30), day_name, font=small_font, fill=color)

    # Draw the days and boxes around them
    row_height = (image_height - 50) / 6  # 6 rows and 50 pixels for header space
    for row_idx in range(6):
        for col_idx, day in enumerate(cal[row_idx] if row_idx < len(cal) else [0]*7):
            color = "red" if col_idx == 0 or col_idx == 6 else "black"
            
            if day != 0:  # 0 indicates no day
                day_str = str(day)
                text_bbox = draw.textbbox((col_idx * col_width, 50 + row_idx * row_height), day_str, font=small_font)
                text_w = text_bbox[2] - text_bbox[0]
                text_h = text_bbox[3] - text_bbox[1]

                # Centering the text in the box
                x_text = col_idx * col_width + (col_width - text_w) / 2
                y_text = 50 + row_idx * row_height + (row_height - text_h) / 2
                draw.text((x_text, y_text), day_str, font=small_font, fill=color)

            # Drawing the box
            draw.rectangle([col_idx * col_width, 50 + row_idx * row_height,
                            (col_idx + 1) * col_width, 50 + (row_idx + 1) * row_height], outline="black")

    # Save the image
    img.save('calendar.png')

draw_calendar(2023, 6)




In [6]:
import calendar
from PIL import Image, ImageDraw, ImageFont
import numpy as np

class CalMonth:
    def __init__(self, year, month):
        self.year = year
        self.month = month
        self.day_colors = {}  # Format: {day: color, ...}
        self.day_bgcolors = {}  # Format: {day: bgcolor, ...}

    def set_day_color(self, day, color):
        """Set the text color for a specific day."""
        self.day_colors[day] = color

    def set_day_bgcolor(self, day, bgcolor):
        """Set the background color for a specific day."""
        self.day_bgcolors[day] = bgcolor

    def draw(self):
        calendar.setfirstweekday(calendar.SUNDAY)
        cal = calendar.monthcalendar(self.year, self.month)
        day_names = list(calendar.day_abbr)[-1:] + list(calendar.day_abbr)[:-1]

        image_width = 350
        image_height = 200
        img = Image.new('RGB', (image_width, image_height), 'white')
        draw = ImageDraw.Draw(img)

        font_path = "/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf"
        font = ImageFont.truetype(font_path, 15)
        small_font = ImageFont.truetype(font_path, 12)

        title = calendar.month_name[self.month] + " " + str(self.year)
        title_bbox = draw.textbbox((0, 0), title, font=font)
        title_w = title_bbox[2] - title_bbox[0]
        title_h = title_bbox[3] - title_bbox[1]
        draw.text(((image_width - title_w) / 2, 5), title, font=font, fill="black")

        col_width = image_width / 7
        for idx, day_name in enumerate(day_names):
            color = "red" if idx == 0 or idx == 6 else "black"
            draw.text((idx * col_width + 10, 35), day_name, font=small_font, fill=color)

        row_height = (image_height - 50) / 6
        for row_idx in range(6):
            for col_idx, day in enumerate(cal[row_idx] if row_idx < len(cal) else [0]*7):
                day_color = self.day_colors.get(day, "red" if col_idx == 0 or col_idx == 6 else "black")
                bgcolor = self.day_bgcolors.get(day, "white")

                if day != 0:
                    day_str = str(day)
                    text_bbox = draw.textbbox((col_idx * col_width, 50 + row_idx * row_height), day_str, font=small_font)
                    text_w = text_bbox[2] - text_bbox[0]
                    text_h = text_bbox[3] - text_bbox[1]

                    x_text = col_idx * col_width + (col_width - text_w) / 2
                    y_text = 50 + row_idx * row_height + (row_height - text_h) / 2
                    
                    # Drawing the background color before drawing the text
                    draw.rectangle([col_idx * col_width, 50 + row_idx * row_height,
                                    (col_idx + 1) * col_width, 50 + (row_idx + 1) * row_height], fill=bgcolor)
                    
                    draw.text((x_text, y_text), day_str, font=small_font, fill=day_color)

                draw.rectangle([col_idx * col_width, 50 + row_idx * row_height,
                                (col_idx + 1) * col_width, 50 + (row_idx + 1) * row_height], outline="black")
        return img
        #img.save('calendar.png')

# Usage:
calendar_month = CalMonth(2023, 6)
calendar_month.set_day_color(14, 'green')
calendar_month.set_day_bgcolor(14, 'yellow')

image = calendar_month.draw()

# Display the image
image.show()


In [20]:
import plotly.graph_objects as go
months = ['Aug-21', 'Sep', 'Oct', 'Nov', 'Dec']
awd_fall = [12, 21, 22, 19, 19]
id_fall = [6, 21, 22, 16, 9]

def create_plotly_table(months: list, awd: list, id: list):
    # Data for the table
    header = ['Fall', 'AWD', 'ID']
    # Final output row
    months.append("Total")
    awd.append(sum(awd))
    id.append(sum(id))

    # Create the table using plotly.graph_objects
    fig = go.Figure(data=[go.Table(
        header=dict(values=header, line = dict(color='black', width=1)),
        cells=dict(values=[
            months,
            awd_fall,
            id_fall,
        ],
        fill_color=[
            ['white'] * 6,  # Fall column background color
            ['#CCECFF'] * 6,   # AWD column background color
            ['#FFFFCC'] * 6,    # ID column background color
        ],
        line = dict(color='black', width=1)
        )
    )])
    fig.update_layout(
        autosize=False,
        width=450
    )

    # Show the table
    fig.show()

# Call the function
create_plotly_table(months, awd_fall, id_fall)


In [14]:
from app.classes.month import DayType
import plotly.graph_objects as go
from plotly.subplots import make_subplots

day_type_list = []
def create_legend_table():
    for day_type in DayType:
        day_type_list.append(day_type.name)
    cell_values = ['Cell 1', 'Cell 2', 'Cell 3', 'Cell 4', 'Cell 5', 'Cell 6', 'Cell 7', 'Cell 8']

    # Define predefined colors for each cell
    cell_colors = ['red', 'green', 'blue', 'yellow', 'orange', 'purple', 'cyan', 'pink']

    # Create the table using plotly.graph_objects
    fig = go.Figure(data=[go.Table(
        header=dict(values=day_type_list),
        cells=dict(values=[cell_values],
                   fill_color=[cell_colors],  # Set cell colors
                   align='center',  # Align text to the center of cells
                   font=dict(color='white', size=14)  # Text color and size
                   ))
    ])

    # Configure the table layout
    fig.update_layout(width=800, height=200)

    # Show the table
    fig.show()

create_legend_table()
      

In [13]:
from app.classes.month import DayType
import plotly.graph_objects as go
from plotly.subplots import make_subplots

day_type_list = []
for day_type in DayType:
    day_type_list.append(day_type.name)
print(day_type_list)

['NONE', 'AWD', 'CONVOCATION', 'ID', 'FINALS', 'NO_CLASS_CAMPUS_OPEN', 'COMMENCEMENT', 'SUMMER_SESSION', 'WINTER_SESSION']
